In [ ]:
from numpy import load, zeros, ones
from numpy.random import randn, randint

In [ ]:
# Load processed faces
data = load("faces.npz")

In [ ]:
data.files

In [ ]:
X = data['arr_0']

In [ ]:
X = X.astype('float16')

In [ ]:
X = (X-127.5)/127.5

In [ ]:
X.shape

In [ ]:
def generate_real_samples(n):
    ix = randint(0, X.shape[0], n)
    x = X[ix]
    y = ones((n, 1))
    return x, y

In [ ]:
def generate_latent_points(latent_dim, n):
    x_input = randn(n*latent_dim)
    x_input = x_input.reshape(n, latent_dim)
    return x_input

In [ ]:
def generate_fake_samples(generator, latent_dim, n):
    x_input = generate_latent_points(latent_dim, n)
    x = generator.predict(x_input)
    y = zeros((n, 1))
    return x, y

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, LeakyReLU, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
features=80

In [ ]:
def define_discriminator(in_shape=(80,80,3)):
    model = Sequential()
    model.add(Conv2D(features, (5,5), padding="same", input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(features, (5,5), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(features, (5,5), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(features, (5,5), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(features, (5,5), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation="sigmoid"))
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
from tensorflow.keras.layers import Conv2DTranspose, Reshape

In [ ]:
def define_generator(latent_dim):
    model = Sequential()
    model.add(Dense(features*5*5, input_dim=latent_dim))
    model.add(Reshape((5,5,features)))
    model.add(Conv2DTranspose(features, (4,4), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(features, (4,4), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(features, (4,4), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(features, (4,4), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(3, (5,5), activation="tanh", padding="same"))
    return model

In [ ]:
def define_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss="binary_crossentropy", optimizer=opt)
    return model

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def save_plot(examples, epoch, n=10):
    plt.figure(figsize=(14,12))
    examples = (examples+1)/2
    for i in range(n*n):
        plt.subplot(n, n, i+1)
        plt.axis('off')
        plt.imshow(examples[i])
    filename = "./samples/Epoch_%03d.png"%(epoch+1)
    plt.savefig(filename)
    plt.close()

In [ ]:
def summarize_performance(epoch, generator, discriminator, latent_dim, n=100):
    X_real, y_real = generate_real_samples(n)
    _, acc_real = discriminator.evaluate(X_real, y_real, verbose=0)
    X_fake, y_fake = generate_fake_samples(generator, latent_dim, n)
    _, acc_fake = discriminator.evaluate(X_fake, y_fake, verbose=0)
    
    print("Accuracy- real: %.0f%%, fake: %.0f%%" % (acc_real*100, acc_fake*100))
    save_plot(X_fake, epoch)
    
    filename = "./models/generator_%03d.h5"%(epoch+1)
    generator.save(filename)

In [ ]:
def train(generator, discriminator, gan, latent_dim, epochs=100, batch=128):
    batch_per_epoch = X.shape[0]//batch
    half_batch = batch_per_epoch//2
    for i in range(epochs):
        for j in range(batch_per_epoch):
            X_real, y_real = generate_real_samples(half_batch)
            d_loss1, _ = discriminator.train_on_batch(X_real, y_real)
            
            X_fake, y_fake = generate_fake_samples(generator, latent_dim, half_batch)
            d_loss2, _ = discriminator.train_on_batch(X_fake, y_fake)
            
            X_gan = generate_latent_points(latent_dim, batch)
            y_gan = ones((batch, 1))
            
            g_loss = gan.train_on_batch(X_gan, y_gan)
            
            print("Epoch %d, %d/%d, d1=%.3f, d2=%.3f, g=%.3f"%
                  (i+1, j+1, batch_per_epoch, d_loss1, d_loss2, g_loss))
        summarize_performance(i, generator, discriminator, latent_dim)

In [ ]:
latent_dim=100

In [ ]:
discriminator = define_discriminator()

In [ ]:
generator = define_generator(latent_dim)

In [ ]:
gan = define_gan(generator, discriminator)

In [ ]:
summarize_performance(-1, generator, discriminator, latent_dim)

In [ ]:
train(generator, discriminator, gan, latent_dim, epochs=20, batch=256)